In [5]:
from datasets import load_dataset, DatasetDict, Dataset
import pandas as pd
import ast
import datasets
from tqdm import tqdm
import time

In [6]:
# load data

data_sample = load_dataset("QuyenAnhDE/Diseases_Symptoms")

Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

In [7]:
data_sample

DatasetDict({
    train: Dataset({
        features: ['Code', 'Name', 'Symptoms', 'Treatments'],
        num_rows: 400
    })
})

In [8]:
updated_data = [{'Name':item['Name'],'Symptoms':item['Symptoms']} for item in data_sample['train']]

In [9]:
df = pd.DataFrame(updated_data)

In [10]:
df.head()

,Name,Symptoms
0,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o..."
1,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue"
2,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck..."
3,Cryptorchidism,"Absence or undescended testicle(s), empty scro..."
4,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala..."


In [11]:
# Extract the symptopms

df['Symptoms'] = df['Symptoms'].apply(lambda x:', '.join(x.split(', ')))

In [12]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split

In [13]:
if torch.cuda.is_available():
  device = torch.device('cuda')

else:
  try:
    device = torch.device('mps')
  except Exception:
    device = torch.device('cpu')

In [14]:
device

device(type='mps')

In [16]:
pip install torch torchvision torchaudio


   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.2 MB 640.0 kB/s eta 0:00:02
   -- ------------------------------------- 0.1/1.2 MB 762.6 kB/s eta 0:00:02
   ------ --------------------------------- 0.2/1.2 MB 1.3 MB/s eta 0:00:01
   ----------- ---------------------------- 0.3/1.2 MB 1.7 MB/s eta 0:00:01
   -------------- ------------------------- 0.4/1.2 MB 1.9 MB/s eta 0:00:01
   --------------------------- ------------ 0.8/1.2 MB 2.8 MB/s eta 0:00:01
   ---------------------------------------  1.2/1.2 MB 3.5 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 3.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   --------- ------------------------------ 0.6/2.4 MB 17.0 MB/s eta 0:00:01
   ----------- ---------------------------- 0.7/2.4 MB 14.2 MB/s eta 0:00:01
   ----------------- ------

In [18]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2').to(device)


In [19]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [20]:
BATCH_SIZE = 8

In [21]:
# Dataset preparation

class LanguageDataset(Dataset):
  def __init__(self,df, tokenizer):
    self.labels = df.columns
    self.data = df.to_dict(orient = 'records')
    self.tokenizer = tokenizer
    x = self.fittest_max_length(df)
    self.max_length = x

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    x = self.data[idx][self.labels[0]]
    y = self.data[idx][self.labels[1]]
    text = f"{x} | {y}"
    tokens = self.tokenizer.encode_plus(text, return_tensors = 'pt', max_length=128, padding = 'max_length', truncation=True)
    return tokens

  def fittest_max_length(self, df):
    """
    Smallest power of two larger than the longest term in the data set.
    Important to set up max length to speed training time.
    """
    max_length = max(len(max(df[self.labels[0]], key=len)), len(max(df[self.labels[1]], key=len)))
    x = 2
    while x < max_length: x = x * 2
    return x

# Cast the Huggingface data set as a LanguageDataset we defined above
data_sample = LanguageDataset(df, tokenizer)

In [22]:
data_sample

In [23]:
# Create train, valid
train_size = int(0.8 * len(data_sample))
valid_size = len(data_sample) - train_size
train_data, valid_data = random_split(data_sample, [train_size, valid_size])

In [24]:
# Make the iterators
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=BATCH_SIZE)

In [25]:
# Set the number of epochs
num_epochs = 10

In [26]:
# Training parameters
batch_size = BATCH_SIZE
model_name = 'distilgpt2'
gpu = 0

In [27]:
# Set the learning rate and loss function
## CrossEntropyLoss measures how close answers to the truth.
## More punishing for high confidence wrong answers
criterion = nn.CrossEntropyLoss(ignore_index = tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=5e-4)
tokenizer.pad_token = tokenizer.eos_token

In [28]:
# Init a results dataframe
results = pd.DataFrame(columns=['epoch', 'transformer', 'batch_size', 'gpu',
                                'training_loss', 'validation_loss', 'epoch_duration_sec'])

In [29]:
# The training loop
for epoch in range(num_epochs):
    start_time = time.time()  # Start the timer for the epoch

    # Training
    ## This line tells the model we're in 'learning mode'
    model.train()
    epoch_training_loss = 0
    train_iterator = tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs} Batch Size: {batch_size}, Transformer: {model_name}")
    for batch in train_iterator:
        optimizer.zero_grad()
        inputs = batch['input_ids'].squeeze(1).to(device)
        targets = inputs.clone()
        outputs = model(input_ids=inputs, labels=targets)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_iterator.set_postfix({'Training Loss': loss.item()})
        epoch_training_loss += loss.item()
    avg_epoch_training_loss = epoch_training_loss / len(train_iterator)

    # Validation
    ## This line below tells the model to 'stop learning'
    model.eval()
    epoch_validation_loss = 0
    total_loss = 0
    valid_iterator = tqdm(valid_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}")
    with torch.no_grad():
        for batch in valid_iterator:
            inputs = batch['input_ids'].squeeze(1).to(device)
            targets = inputs.clone()
            outputs = model(input_ids=inputs, labels=targets)
            loss = outputs.loss
            total_loss += loss
            valid_iterator.set_postfix({'Validation Loss': loss.item()})
            epoch_validation_loss += loss.item()

    avg_epoch_validation_loss = epoch_validation_loss / len(valid_loader)

    end_time = time.time()  # End the timer for the epoch
    epoch_duration_sec = end_time - start_time  # Calculate the duration in seconds

    new_row = {'transformer': model_name,
               'batch_size': batch_size,
               'gpu': gpu,
               'epoch': epoch+1,
               'training_loss': avg_epoch_training_loss,
               'validation_loss': avg_epoch_validation_loss,
               'epoch_duration_sec': epoch_duration_sec}  # Add epoch_duration to the dataframe

    results.loc[len(results)] = new_row
    print(f"Epoch: {epoch+1}, Validation Loss: {total_loss/len(valid_loader)}")

Training Epoch 1/10 Batch Size: 8, Transformer: distilgpt2: 100%|█| 40/40 [04:07<00:00,  6.20s/it, Training Loss=0.698]
Validation Epoch 1/10: 100%|█████████████████████████████████████| 10/10 [00:22<00:00,  2.26s/it, Validation Loss=0.68]


Epoch: 1, Validation Loss: 0.7025867700576782


Training Epoch 2/10 Batch Size: 8, Transformer: distilgpt2: 100%|█| 40/40 [04:21<00:00,  6.55s/it, Training Loss=0.643]
Validation Epoch 2/10: 100%|████████████████████████████████████| 10/10 [00:22<00:00,  2.25s/it, Validation Loss=0.592]


Epoch: 2, Validation Loss: 0.6828542351722717


Training Epoch 3/10 Batch Size: 8, Transformer: distilgpt2: 100%|█| 40/40 [04:15<00:00,  6.38s/it, Training Loss=0.358]
Validation Epoch 3/10: 100%|████████████████████████████████████| 10/10 [00:22<00:00,  2.30s/it, Validation Loss=0.596]


...

Epoch: 8, Validation Loss: 0.8971129655838013


Training Epoch 9/10 Batch Size: 8, Transformer: distilgpt2: 100%|█| 40/40 [06:53<00:00, 10.33s/it, Training Loss=0.0916
Validation Epoch 9/10: 100%|████████████████████████████████████| 10/10 [00:36<00:00,  3.66s/it, Validation Loss=0.783]


Epoch: 9, Validation Loss: 0.9510866403579712


Training Epoch 10/10 Batch Size: 8, Transformer: distilgpt2: 100%|█| 40/40 [06:53<00:00, 10.34s/it, Training Loss=0.085
Validation Epoch 10/10: 100%|███████████████████████████████████| 10/10 [00:36<00:00,  3.67s/it, Validation Loss=0.835]

Epoch: 10, Validation Loss: 0.9878532290458679


In [31]:
input_str = "Kidney Failure"
input_ids = tokenizer.encode(input_str, return_tensors='pt').to(device)

output = model.generate(
    input_ids,
    max_length=20,
    num_return_sequences=1,
    do_sample=True,
    top_k=8,
    top_p=0.95,
    temperature=0.5,
    repetition_penalty=1.2
)

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded_output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Kidney Failure | Severe abdominal or back pain, blood in urine, frequent urination
